In [9]:
#!pip install plotly pandas numpy scipy yfinance scipy.stats scipy.signal plotly.express

#!python.exe -m pip install --upgrade pip


In [10]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
import plotly.express as px
import yfinance as yf



In [23]:

# Define parameters
symbol = "ES=F" # S&P 500-mini Futures (/ES) index symbol
per = "1w" # 4 years period
inter = "1m" # 1 day interval
kde_factor = 0.01 # KDE smoothness factor
prominence = 0.25 # Minimum peak prominence
width = 100 # Maximum peak width in points
start_date = "2020-01-01" # Start date
end_date = "2023-10-31" # End date


In [24]:

# Fetch data from yahoo finance
spx = yf.Ticker(symbol)

df = spx.history(period=per, interval=inter)


ES=F: 1m data not available for startTime=1575064903 and endTime=1701208903. Only 7 days worth of 1m granularity data are allowed to be fetched per request.


In [13]:

df = df.loc[df.index > df.index.max() - pd.DateOffset(years=3)] # Filter for last 2 years
df.drop(columns=["Dividends", "Stock Splits","Open", "High", "Low"], inplace=True) # Drop dividends and stock splits columns


In [20]:

# Plot volume profile
fig = px.histogram(df, x="Close", y="Volume", histfunc="sum", nbins=3500)
fig.show()

# Fit KDE to volume profile
kde = stats.gaussian_kde(df["Close"], weights=df["Volume"], bw_method=kde_factor)
x = np.linspace(df["Close"].min(), df["Close"].max(), 3500)
y = kde(x)

# Plot KDE curve and normalized histogram
fig = px.line(x=x, y=y, labels={"x": "Price", "y": "Density"})
fig.add_histogram(x=df["Close"], y=df["Volume"], histfunc="sum", nbinsx=1200, histnorm="probability density", bingroup=None)
fig.show()


In [15]:

# Find peaks in KDE curve
peaks, properties = signal.find_peaks(y, prominence=prominence*(y.max()-y.min()))

# Plot peaks and their properties
fig = px.line(x=x, y=y, labels={"x": "Price", "y": "Density"})
fig.add_scatter(x=x[peaks], y=y[peaks], mode="markers", marker_color="red")
for i in range(len(peaks)):
    fig.add_vline(x=x[int(properties["left_bases"][i])], line_dash="dash", line_color="green")
    fig.add_vline(x=x[int(properties["right_bases"][i])], line_dash="dash", line_color="green")
    #fig.add_hline(y=properties["prominences"][i], line_dash="dash", line_color="blue")
fig.show()


In [16]:

# Print peak prices and densities
print("Peak prices and densities:")
for i in range(len(peaks)):
    print(f"{x[peaks[i]]:.2f}, {y[peaks[i]]:.6f}")


Peak prices and densities:
3662.41, 0.000990
3765.28, 0.001350
3819.21, 0.001337
3872.14, 0.001808
3900.11, 0.002031
3967.03, 0.001886
4001.98, 0.001879
4032.94, 0.001167
4155.79, 0.002803
4173.77, 0.001788
4298.61, 0.001418
4342.56, 0.002244
4391.49, 0.001973
4521.33, 0.001617
4537.31, 0.001640
4565.28, 0.001484
4596.24, 0.001046
4688.12, 0.000956
